<a href="https://colab.research.google.com/github/mrgarris0n/tier-test/blob/main/tier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Driving range visualization with TIER and OpenRouteService

This notebook is created to give you quick glimpse of the TIER micromobility API and the OpenRouteservice API. Run the cells one by one.

In [ ]:
!pip install ipyleaflet

In [3]:
from ipyleaflet import *
from ipywidgets import Layout
import requests
import json
import random
import time
from configparser import ConfigParser

In [4]:
LAT = 47.4988072
LNG = 19.0502223
TIER_API_URL = "https://platform.tier-services.io"


In [5]:
parser = ConfigParser()
_ = parser.read('test.cfg')

TIER_API_KEY = parser.get('api', 'tier')
ORS_API_KEY = parser.get('api', 'ors')


In [6]:
def clear_map(name):
  for l in map.layers:
    if (type(l).__name__ == name):
      map.remove_layer(l)

In [7]:
def get_vehicles_near_pos(lat, lng, radius):
    url = f"{TIER_API_URL}/v2/vehicle?lat={lat}&lng={lng}&radius={radius}"
    # time.sleep(1.0) # security sleep to prevent API overload
    headers = {'X-Api-Key': TIER_API_KEY}
    results = requests.get(url, headers=headers).json()
    return results

In [8]:
def show_vehicles_on_map(map, vehicledata):
    for vehicle in vehicledata['data']:
        pos = (vehicle['attributes']['lat'], vehicle['attributes']['lng'])
        range = vehicle['attributes']['currentRangeMeters']
        battery = vehicle['attributes']['batteryLevel']
        title = f"Battery Level: {battery}%, range: {range/1000} km"
        
        marker = Marker(location=pos, draggable=False, opacity=battery/100)
        marker.on_click(handle_click)
        marker.title = title
        map.add_layer(marker)

In [9]:
def get_ranges(lat, lng, range):
  loc = [lng, lat]
  
  body = {"locations":[loc],"range":[range],"range_type":"distance"}

  headers = {
      'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
      'Authorization': ORS_API_KEY,
      'Content-Type': 'application/json; charset=utf-8'
  }
  
  call = requests.post('https://api.openrouteservice.org/v2/isochrones/cycling-electric', json=body, headers=headers)

  d = json.loads(call.text)
  return d

In [10]:
def random_color(feature):
    return {
        'color': 'black',
        'fillColor': random.choice(['red', 'yellow', 'green', 'orange']),
    }

In [11]:
def handle_click_map(**kwargs):
  if kwargs.get('type') == 'click':
    lat = kwargs['coordinates'][0]
    lng = kwargs['coordinates'][1]

    clear_map('Marker')
    clear_map('GeoJSON')
    
    vehicles = get_vehicles_near_pos(lat, lng, 1500)
  
    show_vehicles_on_map(map, vehicles)

In [12]:
def handle_click(**kwargs):
    lat = kwargs['coordinates'][0]
    lng = kwargs['coordinates'][1]
    data = {}
    
    clear_map('GeoJSON')
    
    vehicle = get_vehicles_near_pos(lat, lng, 1)
    if vehicle['data'] is not None:
      range = vehicle['data'][0]['attributes']['currentRangeMeters']
      data = get_ranges(lat, lng, range)

      geo_json = GeoJSON(
          data=data,
          style={
              'opacity': 1, 'dashArray': '9', 'fillOpacity': 0.1, 'weight': 1
          },
          hover_style={
              'color': 'white', 'dashArray': '0', 'fillOpacity': 0.5
          },
          style_callback=random_color
          )


      map.add_layer(geo_json)
      



In [13]:
def create_simple_map():
    center = (LAT, LNG)
   
    new_map = Map(center=center, scroll_wheel_zoom=True, zoom=16, layout=Layout(width='95%', height='1200px'))
    
    control = LayersControl(position='topright')
    new_map.add_control(control)
    new_map.on_interaction(handle_click_map)

    return new_map

If you click somewhere on the map, all the TIER vehicles within 1.5 km range will appear. The opacity of the markers is directly proportional to the battery level. You can check the battery level and the driving range of a vehicle by hovering the mouse over their markers. If you click on a vehicle, an isochrone range will be calculated. This means the hypothetic range which the vehicle can travel with the given charge. 

Limitations: 
*   TIER zone constraints and limits are ignored
*   Only Budapest and Siófok is working
*   E-Bike profile from Openroute service were used, e-scooter characteristics may differ 

DISCLAIMER: This is for demonstration purposes only, do not use for real route planning!



In [14]:
map = create_simple_map()
map

Map(center=[47.4988072, 19.0502223], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…